id 부여

In [62]:
import os
import pandas as pd
import mysql.connector

#DB 연결
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',
    database='premo',
)
cursor = conn.cursor()

#team_name 추출
team_set = set()

data_root = './datas'

for folder in os.listdir(data_root):
    folder_path = os.path.join(data_root, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.csv') and 'teams' in file.lower():
                file_path = os.path.join(folder_path, file)
                try:
                    df = pd.read_csv(file_path)
                    
                    if 'common_name' in df.columns:
                        team_set.update(df['common_name'].dropna().unique())
                except Exception as e:
                    print(f"X Error in {file_path}: {e}")
                    
""" #db에 삽입(중복 방지)
for team in team_set:
    cursor.execute("insert into teams (team_name) values (%s)", (team,)) """
    
#완료
conn.commit()
cursor.close()
conn.close()

In [63]:

#matches
matches_drop = ['timestamp', 'date_GMT', 'status', 'attendance', 'referee', 'stadium_name']
goal_columns = ['home_team_goal_timings', 'away_team_goal_timings']
interval_labels = ['0_9', '10_19', '20_29', '30_39', '40_49', '50_59', '60_69', '70_79', '80_89', '90_99', '100_plus']

#goal_timings에서 초 제거
def process_goal_timings(goal_str):
    if pd.isna(goal_str):
        return []
    goal_str = goal_str.replace("'", "")
    return [int(x) for x in goal_str.split(',') if x.strip().isdigit()]

#one-hot encoding 함수
def one_hoe_encode_goals(goal_list):
    bins = [0]*11
    for g in goal_list:
        idx = min(g//10, 10)
        bins[idx] = 1
    return bins
    

for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'matches' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                
                #불필요한 컬럼 삭제
                df_processed = df.drop(columns=[col for col in matches_drop if col in df.columns])
                
                #one-hot encoding 수행
                for team in ['home', 'away']:
                    col = f"{team}_team_goal_timings"
                    if col in df_processed.columns:
                        
                        one_hot_series = df_processed[col].apply(lambda x: one_hoe_encode_goals(process_goal_timings(x)))
                        one_hot_df = pd.DataFrame(one_hot_series.tolist(), columns=[f"{team}_goal_{label}" for label in interval_labels])
                        #기존 컬럼 제거하고 결합
                        df_processed = df_processed.drop(columns=[col])
                        df_processed = pd.concat([df_processed, one_hot_df], axis=1)
                        
                #저장
                new_path = os.path.join(root, file.replace('.csv', '_processed.csv'))
                df_processed.to_csv(new_path, index=False)
                print(f"[완료] matches: {new_path}")
            except Exception as e:
                print(f"[오류] {file_path}: {e}")

[완료] matches: ./datas\1011\england-premier-league-matches-2010-to-2011-stats_processed.csv
[완료] matches: ./datas\1011\england-premier-league-matches-2010-to-2011-stats_processed_processed.csv
[완료] matches: ./datas\1112\england-premier-league-matches-2011-to-2012-stats_processed.csv
[완료] matches: ./datas\1112\england-premier-league-matches-2011-to-2012-stats_processed_processed.csv
[완료] matches: ./datas\1213\england-premier-league-matches-2012-to-2013-stats_processed.csv
[완료] matches: ./datas\1213\england-premier-league-matches-2012-to-2013-stats_processed_processed.csv
[완료] matches: ./datas\1213\~$england-premier-league-matches-2012-to-2013-stats_processed.csv
[완료] matches: ./datas\1213\~$england-premier-league-matches-2012-to-2013-stats_processed_processed.csv
[완료] matches: ./datas\1314\england-premier-league-matches-2013-to-2014-stats_processed.csv
[완료] matches: ./datas\1314\england-premier-league-matches-2013-to-2014-stats_processed_processed.csv
[완료] matches: ./datas\1415\england-p

1. matches - home_goal_timings, away_goal_timings one-hot encoding 처리 후 매핑
2. csv 데이터 id 매핑

In [64]:
#players
players_drop = ['birthday', 'birthday_GMT', 'league', 'season', 'nationality']

for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'players' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                
                #average_rating_overall 이후 컬럼 제거
                if 'average_rating_overall' in df.columns:
                    cut_idx = df.columns.get_loc('average_rating_overall')
                    df = df.iloc[:, :cut_idx+1]
                    
                df_processed = df.drop(columns=[col for col in players_drop if col in df.columns])
                    
                if 'Current Club' in df_processed.columns:
                    df_processed = df_processed.rename(columns={'Current Club': 'team_name'})
                    
                new_path = os.path.join(root, file.replace('.csv', '_processed.csv'))
                df_processed.to_csv(new_path, index=False)
                print(f"[완료] players 컬럼 제거 후 저장: {new_path}")
                
            except Exception as e:
                print(f"[오류] {file_path}: {e}")


[완료] players 컬럼 제거 후 저장: ./datas\1011\england-premier-league-players-2010-to-2011-stats_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1011\england-premier-league-players-2010-to-2011-stats_processed_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1011\england-premier-league-players_avg-2010-to-2011-stats_processed_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1112\england-premier-league-players-2011-to-2012-stats_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1112\england-premier-league-players-2011-to-2012-stats_processed_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1112\england-premier-league-players_avg-2011-to-2012-stats_processed_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1213\england-premier-league-players-2012-to-2013-stats_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1213\england-premier-league-players-2012-to-2013-stats_processed_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1213\england-premier-league-players_avg-2012-to-2013-stats_processed_processed.csv
[완료] playe

In [65]:
#teams

#teams+teams2 병합
for root, dirs, files in os.walk(data_root):
    files_lower = [f.lower() for f in files]
    
    #teams와 teams2 파일이 모두 존재하는 폴더만 처리
    if any('teams' in f for f in files_lower) and any('teams2' in f for f in files_lower):
        try:
            #실제 파일명 찾기
            teams1_file = next(f for f in files if 'teams' in f.lower())
            teams2_file = next(f for f in files if 'teams2' in f.lower())
            
            #파일 경로
            path1 = os.path.join(root, teams1_file)
            path2 = os.path.join(root, teams2_file)
            
            #파일 불러오기
            teams1 = pd.read_csv(path1)
            teams2 = pd.read_csv(path2)
            
            #병합
            teams = pd.merge(teams1, teams2, on='team_name', suffixes=('_t1', '_t2'))
            
            #중복 컬럼 제거
            cols_to_drop = []
            for col in teams.columns:
                if col.endswith('_t1'):
                    base = col[:-3]
                    t2_col = base + '_t2'
                    if t2_col in teams.columns and teams[col].equals(teams[t2_col]):
                        cols_to_drop.append(t2_col)
                        
            teams_clean = teams.drop(columns=cols_to_drop)
            teams_clean.columns = [c.replace('_t1','').replace('_t2','') for c in teams_clean.columns]
            
            #저장 경로
            merged_filename = teams1_file.replace('teams', 'merged_teams')
            save_path = os.path.join(root, merged_filename.replace('.csv', '_processed.csv'))
            teams_clean.to_csv(save_path, index=False)
            print(f"[완료] 병합 및 저장: {save_path}")
        except Exception as e:
            print(f"[오류] {root}: {e}")
       
#삭제     
teams_drop = ['team_name', 'season', 'country']
    
for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'merged_teams' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                
                df = df.drop(columns=[col for col in teams_drop if col in df.columns])
                
                if 'common_name' in df.columns:
                    df = df.rename(columns={'common_name':'team_name'})
                    df.to_csv(file_path, index=False)
                    print(f"[완료] teams: {file_path}")
                else:
                    print(f"[오류] common_name 변경:{file_path}")
            except Exception as e:
                print(f"[오류] {file_path}: {e}")

[오류] ./datas\1011: You are trying to merge on int64 and object columns for key 'team_name'. If you wish to proceed you should use pd.concat
[오류] ./datas\1112: You are trying to merge on int64 and object columns for key 'team_name'. If you wish to proceed you should use pd.concat
[오류] ./datas\1213: You are trying to merge on int64 and object columns for key 'team_name'. If you wish to proceed you should use pd.concat
[오류] ./datas\1314: You are trying to merge on int64 and object columns for key 'team_name'. If you wish to proceed you should use pd.concat
[오류] ./datas\1415: You are trying to merge on int64 and object columns for key 'team_name'. If you wish to proceed you should use pd.concat
[오류] ./datas\1516: You are trying to merge on int64 and object columns for key 'team_name'. If you wish to proceed you should use pd.concat
[오류] ./datas\1617: You are trying to merge on int64 and object columns for key 'team_name'. If you wish to proceed you should use pd.concat
[오류] ./datas\1718: Y

In [66]:
#DB 연결
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',
    database='premo',
)
cursor = conn.cursor()

#team_name -> team_id 매핑
team_df = pd.read_sql("SELECT team_id, team_name FROM teams", conn)
team_map = dict(zip(team_df['team_name'], team_df['team_id']))

#position_name -> position_id 매핑
position_df = pd.read_sql("SELECT position_id, position_name FROM positions", conn)
position_map = dict(zip(position_df['position_name'], position_df['position_id']))

for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'processed' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                
                #matches 파일일 경우
                if 'matches' in file.lower():
                    for col in ['home_team_name', 'away_team_name']:
                        if col in df.columns:
                            df[col] = df[col].map(team_map)
                
                 #players 파일일 경우
                if 'players' in file.lower():
                    if 'position' in df.columns:
                        df['position'] = df['position'].map(position_map)
                
                if 'team_name' in df.columns:
                    df['team_name'] = df['team_name'].map(team_map)
                    
                df.to_csv(file_path, index=False)
                print(f"[완료] id 매핑: {file_path}")
            except Exception as e:
                print(f"[오류] {file_path}: {e}")
           
#완료
conn.commit()
cursor.close()
conn.close()

[완료] id 매핑: ./datas\1011\england-premier-league-final-2010-to-2011-stats_processed.csv

C:\Users\user080524\AppData\Local\Temp\ipykernel_9008\1670578138.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  team_df = pd.read_sql("SELECT team_id, team_name FROM teams", conn)
C:\Users\user080524\AppData\Local\Temp\ipykernel_9008\1670578138.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  position_df = pd.read_sql("SELECT position_id, position_name FROM positions", conn)



[완료] id 매핑: ./datas\1011\england-premier-league-matches-2010-to-2011-stats_processed.csv
[완료] id 매핑: ./datas\1011\england-premier-league-matches-2010-to-2011-stats_processed_processed.csv
[완료] id 매핑: ./datas\1011\england-premier-league-merged_teams-2010-to-2011-stats_processed.csv
[완료] id 매핑: ./datas\1011\england-premier-league-players-2010-to-2011-stats_processed.csv
[완료] id 매핑: ./datas\1011\england-premier-league-players-2010-to-2011-stats_processed_processed.csv
[완료] id 매핑: ./datas\1011\england-premier-league-players_avg-2010-to-2011-stats_processed.csv
[완료] id 매핑: ./datas\1011\england-premier-league-players_avg-2010-to-2011-stats_processed_processed.csv
[완료] id 매핑: ./datas\1112\england-premier-league-final-2011-to-2012-stats_processed.csv
[완료] id 매핑: ./datas\1112\england-premier-league-matches-2011-to-2012-stats_processed.csv
[완료] id 매핑: ./datas\1112\england-premier-league-matches-2011-to-2012-stats_processed_processed.csv
[완료] id 매핑: ./datas\1112\england-premier-league-merged_tea

In [67]:
#팀별 평균 스탯 구하기
for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'players' in file.lower() and 'processed' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                
                if 'position' in df.columns:
                    df = df.drop(columns=['position'])
                    
                numeric_cols = df.select_dtypes(include='number').columns
                numeric_cols = [col for col in numeric_cols if col != 'team_name']
                
                #팀별 평균 계산
                df_team_avg = df.groupby('team_name')[numeric_cols].mean().reset_index()
                
                nan_report = df_team_avg.isna().sum()
                nan_report = nan_report[nan_report > 0]
                
                new_path = os.path.join(root, file.replace('players', 'players_avg'))
                df_team_avg.to_csv(new_path, index=False)
                print(f"[완료] players 컬럼 제거 후 저장: {new_path}")
                
            except Exception as e:
                print(f"[오류] {file_path}: {e}")


[완료] players 컬럼 제거 후 저장: ./datas\1011\england-premier-league-players_avg-2010-to-2011-stats_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1011\england-premier-league-players_avg-2010-to-2011-stats_processed_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1011\england-premier-league-players_avg_avg-2010-to-2011-stats_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1011\england-premier-league-players_avg_avg-2010-to-2011-stats_processed_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1112\england-premier-league-players_avg-2011-to-2012-stats_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1112\england-premier-league-players_avg-2011-to-2012-stats_processed_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1112\england-premier-league-players_avg_avg-2011-to-2012-stats_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1112\england-premier-league-players_avg_avg-2011-to-2012-stats_processed_processed.csv
[완료] players 컬럼 제거 후 저장: ./datas\1213\england-premier-league-players_avg-2012-to-2013-stats_proc

In [68]:
#dataset 3개 math의 team_name 기준으로 병합

for root, dirs, files in os.walk(data_root):
    files_lower = [f.lower() for f in files]
    
    if any('matches' in f and 'processed' in f for f in files_lower) and \
    any('players_avg' in f for f in files_lower) and \
    any('merged_teams' in f for f in files_lower):
        
        try:
            matches_file = next(f for f in files if 'matches' in f.lower() and 'processed' in f.lower())
            players_avg_file = next(f for f in files if 'players_avg' in f.lower())
            teams_file = next(f for f in files if 'merged_teams' in f.lower())
            
            path_matches = os.path.join(root, matches_file)
            path_players = os.path.join(root, players_avg_file)
            path_teams = os.path.join(root, teams_file)
            
            matches = pd.read_csv(path_matches)
            players_avg = pd.read_csv(path_players)
            merged_teams = pd.read_csv(path_teams)
            
            #등장하는 팀만 id 필터링
            team_ids = pd.unique(matches[['home_team_name', 'away_team_name']].values.ravel())
            players_avg_filtered = players_avg[players_avg['team_name'].isin(team_ids)]
            merged_teams_filtered = merged_teams[merged_teams['team_name'].isin(team_ids)]
            
            #병합
            matches = matches.merge(players_avg_filtered.add_prefix('home_'), how='left', left_on='home_team_name', right_on='home_team_name')
            matches = matches.merge(players_avg_filtered.add_prefix('away_'), how='left', left_on='away_team_name', right_on='away_team_name')
            
            matches = matches.merge(merged_teams_filtered.add_prefix('home_'), how='left', left_on='home_team_name', right_on='home_team_name')
            matches = matches.merge(merged_teams_filtered.add_prefix('away_'), how='left', left_on='away_team_name', right_on='away_team_name')
            
            if 'Game Week' in matches.columns:
                matches = matches.drop(columns=['Game Week'])
            
            final_file_name = matches_file.replace('matches', 'final')
            save_path = os.path.join(root, final_file_name)
            matches.to_csv(save_path, index=False)
            print(f"[완료] 최종: {save_path}")
        except Exception as e:
            print(f"[오류] {root}:{e}")
            

[완료] 최종: ./datas\1011\england-premier-league-final-2010-to-2011-stats_processed.csv
[완료] 최종: ./datas\1112\england-premier-league-final-2011-to-2012-stats_processed.csv
[완료] 최종: ./datas\1213\england-premier-league-final-2012-to-2013-stats_processed.csv
[완료] 최종: ./datas\1314\england-premier-league-final-2013-to-2014-stats_processed.csv
[완료] 최종: ./datas\1415\england-premier-league-final-2014-to-2015-stats_processed.csv
[완료] 최종: ./datas\1516\england-premier-league-final-2015-to-2016-stats_processed.csv
[완료] 최종: ./datas\1617\england-premier-league-final-2016-to-2017-stats_processed.csv
[완료] 최종: ./datas\1718\england-premier-league-final-2017-to-2018-stats_processed.csv
[완료] 최종: ./datas\1819\england-premier-league-final-2018-to-2019-stats_processed.csv
[완료] 최종: ./datas\1920\england-premier-league-final-2019-to-2020-stats_processed.csv
[완료] 최종: ./datas\2021\england-premier-league-final-2020-to-2021-stats_processed.csv
[완료] 최종: ./datas\2122\england-premier-league-final-2021-to-2022-stats_proces